In [1]:
import pandas as pd
import numpy as np
from sklearn.ensemble import IsolationForest

In [ ]:
train_data = pd.read_csv("xa.s12.00.mhz.1970-03-26HR00_evid00004_fft_cleaned.csv")

## rel_time(sec) in Mars Datasets

X_train = train_data.drop(columns=['time_abs(%Y-%m-%dT%H:%M:%S.%f)', 'time_rel(sec)', 'label', 'weight'], errors='ignore')

model = IsolationForest(contamination=0.0015, random_state=42)  # contamination value is important
model.fit(X_train)

# unlabeled test data
test_data = pd.read_csv("testxa.s12.00.mhz.1969-12-16HR00_evid00006_fft_cleaned.csv")

X_test = test_data.drop(columns=['time_abs(%Y-%m-%dT%H:%M:%S.%f)', 'time_rel(sec)'], errors='ignore')

anomaly_predictions = model.predict(X_test)

test_data['anomaly'] = np.where(anomaly_predictions == -1, 1, 0)  # -1 anomaly, 1 normal

anomalies = test_data[test_data['anomaly'] == 1]  

if not anomalies.empty:
    anomalies_rel_time = anomalies[['time_rel(sec)', 'anomaly']]  ## rel_time(sec) in Mars Datasets 
    anomalies_rel_time = anomalies_rel_time.reset_index(drop=True)

    # saving anomalies in a csv file
    anomalies_rel_time.to_csv('anomalies_rel_time_test.csv', index=False)

    print(f"detected anomaly points: {len(anomalies)}")
    print("rel_time values of anomaly points:")
    print(anomalies_rel_time)
else:
    print("No anomaly detected.")


In [ ]:
# Deciding on contamination value
import random

contaminations = [random.uniform(0.0005, 0.001) for _ in range(10)]

for c in contaminations:
    model = IsolationForest(contamination=c, random_state=42)
    model.fit(X_train)
    anomaly_predictions = model.predict(X_test)
    
  
    test_data['anomaly'] = np.where(anomaly_predictions == -1, 1, 0)
    
    anomalies = test_data[test_data['anomaly'] == 1]
    print(f"Contamination = {c} predicted anomaly points: {len(anomalies)}")